---

=================================================  
Graded Challenge 3  

Nama  : Akbar Fitriawan  
Batch : HCK-14  

>Progarm ini dibuat untuk melakukan web scraping, Data preparation, Business knowladge, statistics descriptive dan statistics inferential. Kasusnya adalah ingin nenambah pendapatan dengan berjualan namun modal produksi tidak mencukupi dan hanya mampu untuk promosi sehingga menjalankan dropshipping di tokopedia. akan tetapi masih bingung dengan ide usaha untuk berjualan, namun teringat yang sedang trend Seblak tetapi  tidak yakin apakah benar bahwa masyarakat memiliki animo yang besar terhadap seblak
=================================================  


# Set up

In [2]:
from selenium import webdriver 
from bs4 import BeautifulSoup
from scipy import stats
import pandas as pd
import numpy as np


# A. Web Scraping

---

In [250]:
# inisialisasi webdriver chrome
driver = webdriver.Chrome()

## Pengambilan Data

In [251]:
# membuat data list penampung

nama = []
harga = []
penjual = []
lokasi_toko = []
terjual = []
rating = []

# looping url sampai halaman

for i in range(1,16):

    # membuat target url dan mengakses dengan webdriver chrome
    url = 'https://www.tokopedia.com/search?navsource=&page={}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st='.format(i)
    driver.get(url)

    # mengambil page source
    html = driver.page_source
    # inisialisasi Beatifulsoup
    soups = BeautifulSoup(html, 'html.parser')
    
    # inisialisasi parent div
    soup = soups.find('div',{"data-testid":"divSRPContentProducts"})

    # looping cari di setiap card
    for item in soup.find_all('div', {'data-testid': 'master-product-card'}):

        # handling eror jika data kosong dan di loop
        try:
            element_nama = item.find('div', {'data-testid':'spnSRPProdName'})
            nama.append(element_nama.get_text())
        except:
            nama.append('0') 
                
        try:
            element_harga = item.find('div', {'data-testid':'spnSRPProdPrice'})
            harga.append(element_harga.get_text())

        except:
            harga.append('0')
                
        try:
            element_lokasi = item.find('span', {'data-testid':'spnSRPProdTabShopLoc'})
            lokasi_toko.append(element_lokasi.get_text())
        except: 
            lokasi_toko.append('0')

        try:
            element_penjual = item.find('span', {'class':'prd_link-shop-name css-1kdc32b flip'})
            penjual.append(element_penjual.get_text())
        except:
            penjual.append('0')

        try:
            element_rating = item.find('span', {'class':'prd_rating-average-text css-t70v7i'})
            rating.append(element_rating.get_text())
            
        except:
            rating.append('0')
            
        try:
            element_terjual = item.find('span', {'class':'prd_label-integrity css-1sgek4h'})
            terjual.append(element_terjual.get_text())
                
        except:
            terjual.append('0')


data = pd.DataFrame()
data['nama'] = nama
data['harga'] = harga
data['penjual'] = penjual
data['lokasi_toko'] = lokasi_toko
data['terjual'] = terjual
data['rating'] = rating


data


,nama,harga,penjual,lokasi_toko,terjual,rating
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp17.000,Central Seblak Nusantara,Tangerang Selatan,2rb+ terjual,4.9
1,Seblak Beton 1kg Pedas Daun Jeruk / Seblak Ker...,Rp42.500,D&R-9 collection,Bandung,0,0
2,GFS Seblak Basah Instan Original - Tinggal Sed...,Rp16.618,Lakoca Official Shop,Cimahi,100+ terjual,4.9
3,Seblak instant murah,Rp4.900,dapurbulbin,Bandung,0,0
4,3 BASO ACI FREE SEBLAK SAJODO SNACK & FOOD,Rp61.999,Sajodo Snack & Food,Tasikmalaya,40+ terjual,5.0
...,...,...,...,...,...,...
145,"baso aci keju, boci, bakso aci, cuanki, seblak...",Rp10.000,baso aci masabi,Jakarta Selatan,100+ terjual,5.0
146,Seblak Bantat 250 gram,Rp15.500,Kriuk Kriuk Snack Kiloan New,Bandung,20 terjual,4.9
147,Seblak Instan Enak dan Murah,Rp11.000,Aishakira online,Jakarta Timur,80+ terjual,5.0
148,cuanki pangsit segitiga atau cipet isi 40 pcs ...,Rp13.999,CUANKI CULAMETAN,Kab. Garut,100+ terjual,4.8


Catatan : Ada beberapa kolom teridentifikasi 0 atau null(Missing value) dikarenakan produk tersebut baru atau lunching  
Solusi : melakukan pengahapusan, namun apa bila informasi ini berguna kemungkinan memasukan ke dataframe lain 

In [252]:
# cek panjang data apakah sama panjang
print(len(data['nama'].values)) #  kolom nama
print(len(data['harga'].values)) # kolom harga
print(len(data['penjual'].values)) # kolom penjual
print(len(data['lokasi_toko'].values)) # kolom lokasi
print(len(data['terjual'].values)) # kolom terjual
print(len(data['rating'].values)) # kolom rating

150
150
150
150
150
150


In [ ]:
# save data agar tidak hilang
data.to_csv('data_scrap.csv', index=False)

---


# B. Persiapan Data

## Load Data

In [3]:
# load Data
df = pd.read_csv('data_scrap.csv')
df.head()

,nama,harga,penjual,lokasi_toko,terjual,rating
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp17.000,Central Seblak Nusantara,Tangerang Selatan,2rb+ terjual,4.9
1,Seblak Beton 1kg Pedas Daun Jeruk / Seblak Ker...,Rp42.500,D&R-9 collection,Bandung,0,0.0
2,GFS Seblak Basah Instan Original - Tinggal Sed...,Rp16.618,Lakoca Official Shop,Cimahi,100+ terjual,4.9
3,Seblak instant murah,Rp4.900,dapurbulbin,Bandung,0,0.0
4,3 BASO ACI FREE SEBLAK SAJODO SNACK & FOOD,Rp61.999,Sajodo Snack & Food,Tasikmalaya,40+ terjual,5.0


---

- Cek informasi data

In [4]:
df.head()

,nama,harga,penjual,lokasi_toko,terjual,rating
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp17.000,Central Seblak Nusantara,Tangerang Selatan,2rb+ terjual,4.9
1,Seblak Beton 1kg Pedas Daun Jeruk / Seblak Ker...,Rp42.500,D&R-9 collection,Bandung,0,0.0
2,GFS Seblak Basah Instan Original - Tinggal Sed...,Rp16.618,Lakoca Official Shop,Cimahi,100+ terjual,4.9
3,Seblak instant murah,Rp4.900,dapurbulbin,Bandung,0,0.0
4,3 BASO ACI FREE SEBLAK SAJODO SNACK & FOOD,Rp61.999,Sajodo Snack & Food,Tasikmalaya,40+ terjual,5.0


ada beberapa kolom nol

In [5]:
# ringakasn data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nama         150 non-null    object 
 1   harga        150 non-null    object 
 2   penjual      150 non-null    object 
 3   lokasi_toko  150 non-null    object 
 4   terjual      150 non-null    object 
 5   rating       150 non-null    float64
dtypes: float64(1), object(5)
memory usage: 7.2+ KB


Catatan:  
- data memiliki 150 baris dan kolom 6
- tipe data memiliki 1 float dan 5 object
- untuk kolom yaitu nama, harga, penjual, terjual, rating
- untuk harga kita rubah menjadi flaot dilangkah selanjutnya
- sama juga untuk tipe data terjual kita rubah menjadi int 

In [6]:
print('baris =', df.shape[0])
print('kolom =',df.shape[1])

baris = 150
kolom = 6


- cek baris dupilkat

In [7]:
df.duplicated().sum().sum()

27

27 data duplikat sehingga harus drop

In [8]:
df = df.drop_duplicates(keep='first')

In [9]:
df.duplicated().sum().sum()

0

In [10]:
df.shape

(123, 6)

hasil memiliki baris 123 data

- cek missing value

In [11]:
df.isnull().sum()

nama           0
harga          0
penjual        0
lokasi_toko    0
terjual        0
rating         0
dtype: int64

tidak ada mising value namun ada beberapa kolom adalah nol yang berarti seller baru atau produk belum terjual

- Data cleaning

In [12]:
df.head()

,nama,harga,penjual,lokasi_toko,terjual,rating
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp17.000,Central Seblak Nusantara,Tangerang Selatan,2rb+ terjual,4.9
1,Seblak Beton 1kg Pedas Daun Jeruk / Seblak Ker...,Rp42.500,D&R-9 collection,Bandung,0,0.0
2,GFS Seblak Basah Instan Original - Tinggal Sed...,Rp16.618,Lakoca Official Shop,Cimahi,100+ terjual,4.9
3,Seblak instant murah,Rp4.900,dapurbulbin,Bandung,0,0.0
4,3 BASO ACI FREE SEBLAK SAJODO SNACK & FOOD,Rp61.999,Sajodo Snack & Food,Tasikmalaya,40+ terjual,5.0


- cek data seller baru atau yang nilai penjualan 0

In [13]:
data_seller_baru = df[df['terjual'] == '0']
print(data_seller_baru.shape[0], 'Data')
data_seller_baru

7 Data


,nama,harga,penjual,lokasi_toko,terjual,rating
1,Seblak Beton 1kg Pedas Daun Jeruk / Seblak Ker...,Rp42.500,D&R-9 collection,Bandung,0,0.0
3,Seblak instant murah,Rp4.900,dapurbulbin,Bandung,0,0.0
21,Cuanki lidah Toping baso aci seblak isi 5 pcs,Rp3.490,Grosir Putra Bdg,Kab. Bandung,0,0.0
30,SEBRING snack seblak kering kriuk,Rp26.000,snack darling,Kab. Bandung Barat,0,0.0
60,Seblak Kerupuk,Rp12.000,agen produk organik jkt,Jakarta Timur,0,0.0
62,Kerupuk Seblak Kering seblak Daun Jeruk dan se...,Rp15.000,Amelia jaya mart,Kab. Tangerang,0,0.0
137,Seblak instan Murah Enak,Rp3.800,Kedai_boci,Semarang,0,0.0


insight: Dilihat dari data tersebut penjual baru atau produk belum terjual diluar jabodetabek lebih banyak 

In [14]:
# drop data penjualan nol
df.drop(df[df.values == '0'].index, inplace=True)

In [15]:
# hapus tanda 'Rp' diubah jadi float
df['harga'] = df['harga'].str.replace(pat='Rp', repl='', regex=True).astype(float)

In [16]:
# hapus tanda yang tidak diperlukan di kolom terjual
df['terjual'] = df['terjual'].str.replace('rb', '000').str.replace('+', '').str.replace('terjual', '')
df['terjual'] = df['terjual'].astype(int)

In [17]:
df.head()

,nama,harga,penjual,lokasi_toko,terjual,rating
0,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,17.000,Central Seblak Nusantara,Tangerang Selatan,2000,4.9
2,GFS Seblak Basah Instan Original - Tinggal Sed...,16.618,Lakoca Official Shop,Cimahi,100,4.9
4,3 BASO ACI FREE SEBLAK SAJODO SNACK & FOOD,61.999,Sajodo Snack & Food,Tasikmalaya,40,5.0
5,"Seblak Rafael, Seblak Coet Instan Halal",25.000,Brother Meat Shop,Depok,250,5.0
6,"Kylafood Seblak Rempah Autentik 115gr, Seblak ...",9.900,Brother Meat Shop,Depok,100,4.9


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nama         116 non-null    object 
 1   harga        116 non-null    float64
 2   penjual      116 non-null    object 
 3   lokasi_toko  116 non-null    object 
 4   terjual      116 non-null    int32  
 5   rating       116 non-null    float64
dtypes: float64(2), int32(1), object(3)
memory usage: 5.9+ KB


> ### Data sudah dibersihkan dan dirubah tipe datanya sesuai konteks data

## Ringkasan informasi data
> - Memiliki 116 baris dan 6 kolom 
> - Memiliki type data 2 float, 1 int dan 3 object
> - data 0 di drop


# C. Memahami Bisnis Atau Problem Statement

---

## Problem Statement

> saya ingin menambah pendatapatan dengan pejualan, namun biaya modal tidak mencukupi sehingga memilih skema dropship di toko online tokopedia.  
sebagai dropshipper seblak yang sedang trend akan tetapi ragu apakah dapat menguntungkan atau tidak.

## SMART

- Spesific
> Mencari tahu potensi penjualan produk seblak di berbagai merchant dalam hal keuntungan dan kerugian di setiap bulan


- Measurable
> Menargetkan penjualan produk >= 50 disetiap bulan 

- Achievable
> melakukan analisa disetiap marchant dengan men-scrapping data di tokopedia


- Relevant
> mendapatkan tambahan penghasilan di tiap bulan tanpa harus mengeluarkan modal yang besar karena dropship  
dan menghilangkan keraguan dalam menjalankan toko

- Time-bound
> Menetapkan target waktu dalam 3 minggu untuk menganalisa dan menjalankan toko di minggu berikutnya

- Ringakasan
> meningkatkan pendapatan dengan berjualan seblak di tokopedia dalam skema dropship.  
 dengan setiap bulannya penjualan >= 50 dan menetapkan target menjalankan toko dalam 4 minggu

# D. Analisis Data

---

- ### Menghitung rata-rata, median, dan standar deviasi dari kolom harga, kolom terjual, dan kolom rating

In [19]:
df[['harga', 'terjual', 'rating']].agg(['mean', 'median', 'std'])

,harga,terjual,rating
mean,23.197862,566.094828,4.814655
median,15.250000,100.000000,4.900000
std,25.914500,1581.454790,0.507724


> - pada kolom harga memiliki rata-rata harga Rp 23.197, median Rp 15.250, dan std 25.9.  
insight : dilihat dari nilai std dan rata-rata hampir mendekati std data lebih condong ke std

> - pada kolom terjual memiliki rata-rata 566 produk, median 100, dan std 1581.4  
insight: menyimpulkan bahwa variasi yang sangat besar

> - pada kolom rating memiliki rata-rata 4.8, median 4.9, dan std 0.5  
insight: dapat disimpulkan bahawa variasi nya kecil, data lebih condong ke median dan rata-rata

- ### Skewness 

In [20]:
# buat series data skewnes
skewnes = df[['harga','terjual','rating']].skew()
skewnes.to_frame(name='skewnes')

,skewnes
harga,2.849512
terjual,4.599239
rating,-7.982875


>Insight:
>- Skewness pada kolom harga menunjukan bahwa distribusi `positive Skew` dan merupakan highly skewed. skewness > 1

>- Skewness pada kolom terjual menunjukan bahwa distribusi `positive Skew`sama highly skewed. skewness > 1

>- berbeda dengan dua kolom di atas ini termasuk `negative Skewed` dan merupakan highly skewed . skewness> -1

>- dari data di atas semuanya highly skew dan data distribusi lebih condong ke kiri karane -7.9 adalah ekor terbesarnya sehingga di sebut `Negative skew`

> catatan outlier nya besar


- ### Kurtosis 

In [21]:
# buat series kurtosis
kurtos = df[['harga','terjual','rating']].kurtosis()
kurtos.to_frame(name='kurtosis')

,kurtosis
harga,8.895571
terjual,23.176418
rating,72.577019


Insight:

> - kolom harga memiliki nilai tinggi sehingga ini termasuk `leptokurtik`
> - kolom terjual juga sama memiliki nilai tinggi termasuk `leptokurtik`
> - kolom rating sama nilai tinggi termasuk `leptokurtik `  
>memiliki outlier cukup besar

In [22]:
# membuat tabel Statistik
stat = df.describe()
# skew dan kurto di gabung
normTestFrame = pd.DataFrame([skewnes,kurtos]).rename(index={0:'skewness', 1:'kurtosis'})
# modus
mode = df[['harga','terjual','rating']].mode().rename(index={0:'modus'})

stat = pd.concat([stat,normTestFrame,mode],ignore_index=False)

stat

,harga,terjual,rating
count,116.000000,116.000000,116.000000
mean,23.197862,566.094828,4.814655
std,25.914500,1581.454790,0.507724
min,2.340000,1.000000,0.000000
25%,8.945000,25.750000,4.800000
50%,15.250000,100.000000,4.900000
75%,26.193750,250.000000,5.000000
max,145.000000,10000.000000,5.000000
skewness,2.849512,4.599239,-7.982875
kurtosis,8.895571,23.176418,72.577019


## Ringkasan 
- jumlah data 116
- rata - rata di kolom harga `23rb` , rata-rata di kolom terjual` 566 `produk, dan untuk rating` 4.8`
- untuk standar deviasi kolom terjual cukup besar data sebarannya berbeda dengan rating yang memiliki data cukup serupa
- harga maksimum` 145.000 `harga minimum `2.340 `
- penjualan paling tinggi `10000` dan penjualan terkecil `1`
- Untuk Skewness data lebih condong ke kiri artinya `Negative skew` dan kolom rating memiliki data dominan
- kurtosis memiliki nilai tinggi artinya `leptokurtic` sehingga data dominan di rating juga
- modus harga adalah 5000, penjualan 100, dan rating 5.0

## Test Normalitas

##### D’Agostino’s K^2   

Catatan: Jika p-value lebih dari 0.05, data memiliki distribusi normal

In [23]:
hargatest = stats.normaltest(df['harga']).pvalue
terjualtest = stats.normaltest(df['terjual']).pvalue
ratingtest = stats.normaltest(df['rating']).pvalue


print(f'p-value harga: {hargatest}')
print(f'p-value terjual: {terjualtest}')
print(f'p-value rating: {ratingtest}')


p-value harga: 3.490996153694984e-20
p-value terjual: 1.609242786092678e-31
p-value rating: 9.369690606739295e-47


Dalam D’Agostino’s K^2 Test, semua data memiliki distribusi normal

#### Shapiro-Wilk Test  
Catatan: Jika p-value lebih dari 0.05, data memiliki distribusi normal

In [24]:
hargatest = stats.shapiro(df['harga']).pvalue
terjualtest = stats.shapiro(df['terjual']).pvalue
ratingtest = stats.shapiro(df['rating']).pvalue

print(f'p-value harga: {hargatest}')
print(f'p-value terjual: {terjualtest}')
print(f'p-value rating: {ratingtest}')

p-value harga: 4.547147723052468e-15
p-value terjual: 5.117685024274034e-20
p-value rating: 3.70940623477274e-21


Dalam Shapiro Wilk test, semua data memiliki distribusi normal

#### Cek outlier  
karena skewness lebih dari 0.5, maka pencarian outlier menggunakan Tukey's rule

#### Tukey's Rule

In [25]:
# Mengambil data quantile 25% dan 75% di kolom harga, terjual dan rating 
q1 = stat.T['25%']
q3 = stat.T['75%']
iqr = q3-q1

# mencari upper boundary dan lower boundary
upperBound = q3 + 1.5*iqr
lowerBound = q3 - 1.5*iqr

# outlier
outlier_harga = df['harga'][(df['harga']<lowerBound['harga'])|(df['harga']>upperBound['harga'])].copy()
outlier_terjual = df['terjual'][(df['terjual']<lowerBound['terjual'])|(df['terjual']>upperBound['terjual'])].copy()
outlier_rating = df['rating'][(df['rating']<lowerBound['rating'])|(df['rating']>upperBound['rating'])].copy()



# persentase outlier
persen_harga = len(outlier_harga)*100/len(df['harga'])
persen_terjual = len(outlier_terjual)*100/len(df['terjual'])
persen_rating = len(outlier_rating)*100/len(df['rating'])

print(f'Persen Harga outliers: {persen_harga:.2f}%')
print(f'persen terjual outliers: {persen_terjual:.2f}%')
print(f'persen rating outliers: {persen_rating:.2f}%')

Persen Harga outliers: 7.76%
persen terjual outliers: 17.24%
persen rating outliers: 8.62%


Dilihat dari data di atas penjualan atau kolom terjual memiliki outlier terbesar di ikuti rating dan harga yang hampir sama,  
artinya penjualan memilliki perbedaan data signifikan dengan rating dan harga

## Melihat Potensi penjualan Seblak

mengambil sempel dari data terjual dengan confidence level = 95%

### Mencari Lower Bound dan Upper Bound

In [26]:
std = df['terjual'].std()
N = len(df['terjual'])

low_terjual, up_terjual = stats.norm.interval(0.95, loc=df['terjual'].mean(),scale=std/np.sqrt(N))

print(f'lower_bound terjual : {round(low_terjual)}')
print(f'upper_bound terjual : {round(up_terjual)}')

lower_bound terjual : 278
upper_bound terjual : 854


Dari data tersebut, dapat diketahui estimasi keuntungan dalam penjualan seblak berkisar dari 278 sampai 854 produk terjual.


# Perbandingan Harga di Jabodetabek dan di Luar Jabodetabek

## Hipotesis test


Dikatakan bahwa ada kemungkinan harga seblak di jabodetabek dan di luar jabodetabek berbeda dikarenakan perbedaan harga bahan baku pembuatannya  


Hipotesis:
- H0: Harga seblak di Jabodetabek dan di luar Jabodetabek sama (rata-rata1 == rata-rata2)
- H1: Harga seblak di Jabodetabek dan di luar Jabodetabek Berbeda (rata-rata1 != rata-rata2)  


Metode:
- Pengujian Langsung
- Pengujian membandingkan rata-rata
- Pengujian Single Sample T-Test


### Membuat data Jabodetabek dan luar Jabodetabek



In [27]:
# list data Jabodetabek 
lokasi = ['jakarta','bogor','depok','tangerang','bekasi']

# Membuat data kosong untuk harga di Jabodetabek dan di luar Jabodetabek
dataLok = []
dataLuar = []


# Loop melalui setiap baris dalam DataFrame
for index, row in df.iterrows():
    # Ubah lokasi toko menjadi lowercase untuk pencocokan yang tidak peka terhadap huruf besar kecil
    lokasi_toko_lower = row['lokasi_toko'].lower()
    # Cek apakah lokasi toko ada dalam list lokasi
    if any(lok in lokasi_toko_lower for lok in lokasi):
        # Jika ada, tambahkan data ke dataLok
        dataLok.append([row['harga'], row['lokasi_toko']])
    else:
        # Jika tidak, tambahkan data ke dataLuar
        dataLuar.append([row['harga'], row['lokasi_toko']])


# membuat frame data perbedaan jobodetabek dan luar jabodetabek
datalok = pd.DataFrame(dataLok).rename(columns={0:'harga',1:'jabodetabek'})
dataluar = pd.DataFrame(dataLuar).rename(columns={0:'harga',1:'di_luar_jabodetabek'})



# cek panjang data dari perbdaan wilayah
[len(dataLok),len(dataLuar)]


[58, 58]

Di lihat dari panjang data di wilayah jabodetabek dan diliaur wilayah jabodetabek sama

In [28]:
# modus data harga jabodetabek
datalok['harga'].mode()

0    11.0
1    15.0
2    18.0
Name: harga, dtype: float64

mengingat data harga jobodetabek lebih konsisten

In [29]:
# modus harga di luar jabodetabek
dataluar['harga'].mode()

0    5.0
Name: harga, dtype: float64

harga di luar jabodetabek cukup bervariasi yang sering muncull disini 5000

In [30]:
# rata-rata harga di jabodetek
datalok['harga'].mean()

19.385827586206894

In [31]:
# rata-rata harga di luar jabodetabek 
dataluar['harga'].mean()

27.009896551724136

In [32]:
print(datalok['harga'].max())
print(datalok['harga'].min())

145.0
3.1


harga max dan min dijabodetabek sangat besar perbadaan nya

In [33]:
print(dataluar['harga'].max())
print(dataluar['harga'].min())

127.88
2.34


sama dengan harga max dan min besar perbedaanya

---
### Pengujian hipotesis secara langsung

In [34]:
# Mencari harga sama dan harga berbeda
harga_sama = datalok['harga'].values == dataluar['harga'].values
harga_beda = datalok['harga'].values != dataluar['harga'].values
# Mencari harga yang kurang dan lebih dari jabodetabek
jabode = datalok['harga'].values > dataluar['harga'].values
luar_jabode = datalok['harga'].values < dataluar['harga'].values

print(f"""
      Banyak barang yang harganya sama di antara jabodetabek dan luar jabodetabek: {harga_sama.sum()}
      
      Banyak barang yang harganya berbeda di antara jabodetabek dan luar jabodetabek: {harga_beda.sum()}
      
      Banyak barang yang mana harga di jabodetabek lebih dari luar jabodetabek: {jabode.sum()}
      
      Banyak barang yang mana harga di jabodetabek kurang dari luar jabodetabek: {luar_jabode.sum()}
      """
    )


# cek dataframe
# datalok
# dataluar


      Banyak barang yang harganya sama di antara jabodetabek dan luar jabodetabek: 2
      
      Banyak barang yang harganya berbeda di antara jabodetabek dan luar jabodetabek: 56
      
      Banyak barang yang mana harga di jabodetabek lebih dari luar jabodetabek: 22
      
      Banyak barang yang mana harga di jabodetabek kurang dari luar jabodetabek: 34
      


#### Dapat di simpulkan H1 benar, karena harga berbeda diantara jabodetabek dan di luar jabodetabek. dari segi harga di luar jobodetabek lebih tinggi 

---
### Pengujian hipotesis dengan membandingkan rata-rata

In [35]:
# Mencari data statistik dan mengoutputnya

mean_lok = np.mean(datalok['harga']) # harga di Jabodetabek
mean_luar = np.mean(dataluar['harga']) # harga di luar Jabodetabek
N_lok = len(datalok)
N_luar = len(dataluar)
std_lok = np.std(datalok['harga'])
std_luar = np.std(dataluar['harga'])

print(f'''
      Rata-rata harga seblak di Jabodetabek: {mean_lok},
      
      Rata-rata harga seblak di luar Jabodetabek: {mean_luar},

      Banyaknya seblak di Jabodetabek: {N_lok},

      Banyaknya seblak di Luar Jabodetabek: {N_lok},

      Simpangan baku harga seblak di Jabodetabek: {std_lok},

      Simpangan baku harga di Luar Jabodetabek: {std_luar},

      H0 Terpenuhi : {mean_lok == mean_luar},

      H1 Terpenuhi : {mean_lok != mean_luar}
      '''
      )



      Rata-rata harga seblak di Jabodetabek: 19.385827586206894,
      
      Rata-rata harga seblak di luar Jabodetabek: 27.009896551724136,

      Banyaknya seblak di Jabodetabek: 58,

      Banyaknya seblak di Luar Jabodetabek: 58,

      Simpangan baku harga seblak di Jabodetabek: 24.164150908074642,

      Simpangan baku harga di Luar Jabodetabek: 26.806241050642676,

      H0 Terpenuhi : False,

      H1 Terpenuhi : True
      


#### Terlihat bahwa rata-rata harga seblak di jabodetabek dan rata-rata harga seblak di Luar Jabodetabek tidak sama.  
#### Dapat disimpulkan bahwa H1 benar.


---
### Pengujian Single Sample T-Test

Dalam Data ini, membuat sample acak data yang digunakan, dengan Looping sampai 100 kali untuk melihat dari setiap pengujian yang mana yang mengeluarkan p-value kurang dari 0.05. dengan begini, data menjadi lebih konsisten dalam hasil pengujian

In [36]:
# Ambil alpha (tingkat signifikansi)
alpha = 0.05
# counter yang di tolak
reject_count = 0


for i in range(100):
    # ambil sample secara acak di jabodetabek dengan ukuran sama
    sample = np.random.choice(datalok['harga'].values, size=len(dataluar['harga'].values), replace=False)

    # uji t-sample satu arah
    t_stat, p_value = stats.ttest_1samp(sample, np.mean(dataluar['harga'].values))

    if p_value < alpha:
        reject_count += 1


# stat dan p_val akhir yang eksekusi
print("t-statistic:", t_stat)
print("p-value:", p_value)

print('Jumlah uji yang menolak H0 : ', reject_count)


t-statistic: -2.382060045548997
p-value: 0.020575843731595253
Jumlah uji yang menolak H0 :  100


Data di atas menunjukan H1 benar dengan mengubah sample secara acak dan men-loop data 100 kali dan menghitung yang menolak H0 

#### `Konklusi` dari semua pengujian test dapat di simpulkan harga di jobodetabek dan harga di luar jabodetabek berbeda.  
#### rata-rata di luar jabodetabek adalah Rp27.00 sedangkan rata-rata di jabodetabek adalah Rp19.38

### Uji Kolerasi

In [38]:
cor_pear, p_valPear = stats.pearsonr(df['harga'], df['rating'])
cor_kendell, p_valKendell = stats.kendalltau(df['harga'], df['rating'])

# Tampilkan hasil uji
print("Nilai korelasi pearson:", cor_pear)
print("Nilai korelasi kandell:", cor_kendell)
print("p-value:", p_valPear)
print("p-value:", p_valKendell)

if p_valPear < 0.05:
    print("Ada hubungan yang signifikan antara harga produk dan rating.")
else:
    print("Tidak ada hubungan yang signifikan antara harga produk dan rating.")

if p_valKendell < 0.05:
    print("Ada hubungan yang signifikan antara harga produk dan rating.")
else:
    print("Tidak ada hubungan yang signifikan antara harga produk dan rating.")

Nilai korelasi pearson: 0.07808942180039297
Nilai korelasi kandell: 0.08000315612393631
p-value: 0.4047241134969272
p-value: 0.2523252997708769
Tidak ada hubungan yang signifikan antara harga produk dan rating.
Tidak ada hubungan yang signifikan antara harga produk dan rating.


dapat disimpulkan bahwa tidak ada kolerasi antara harga produk dan rating dengan melihat p_value < alpha

### Analisa pembelian terhadap produk murah

Dalam hal ini, saya berpikir apakah ada hubungan terjualnya suatu barang terhadap produk murah. Dengan pearson dan kendell

### Pembuatan Hipotesis

Dalam data produk seblak, saya berasumsi bahwa harga suatu produk memiliki hubungan dengan banyaknya pelanggan.

Data yang dibutuhkan:
- Harga
- Terjual

Hipotesis:
- H0:Harga suatu produk tidak ada hubungannya dengan banyaknya pelanggan yang akan membeli
- H1:Harga suatu produk ada hubungannya dengan banyaknya pelanggan yang akan membeli

Metode:
> pearson dan kendell

Uji korelasi antara harga dan terjual

In [39]:

# Menghitung koefisien korelasi Pearson
pearson_corr, _ = stats.pearsonr(df['harga'], df['terjual'])

# Menghitung koefisien korelasi Kendall
kendall_corr, _ = stats.kendalltau(df['harga'], df['terjual'])

# Menentukan apakah nilai korelasi positif atau negatif
if pearson_corr > 0:
    pearson_corr_type = "positif"
elif pearson_corr < 0:
    pearson_corr_type = "negatif"
else:
    pearson_corr_type = "tidak ada korelasi"

if kendall_corr > 0:
    kendall_corr_type = "positif"
elif kendall_corr < 0:
    kendall_corr_type = "negatif"
else:
    kendall_corr_type = "tidak ada korelasi"

# Menampilkan hasil
print("Koefisien korelasi Pearson:", pearson_corr)
print("Tipe korelasi Pearson:", pearson_corr_type)

print("Koefisien korelasi Kendall:", kendall_corr)
print("Tipe korelasi Kendall:", kendall_corr_type)

Koefisien korelasi Pearson: -0.04678862124316436
Tipe korelasi Pearson: negatif
Koefisien korelasi Kendall: -0.03925298191188876
Tipe korelasi Kendall: negatif


oke dari sini dapat disimpulakan bahwa hubungan korelasi rendah dan artinya H0 benar

# E. Kesimpulan

---

- Harga Seblak yang sering muncul di jabodetabek Rp11.000, Rp15.000, dan Rp18.000. saya bisa memulai dengan harga kurang dari Rp19.000
- Mengingat data boudary di atas saya akan mendapat keuntungan penjualan seblak berkisar dari 278 sampai 854 produk terjual perbulan.
- Jabodetabek dan Luar Jabodetabek memiliki perbedaan harga seblak dikarenakan biaya bahan bakunya berbeda
- Jabodetabek lebih murah dibanding Luar Jabodetabek dengan rata-rata harga seblak Jabodetabek < Luar Jabodetabek
- Murah mahalnya harga sangat mempengaruhi banyaknya penjualan yang akan datang
- **Berdasarkan data potensi keuntungan, saya bisa memulai dropshipper seblak yang tadi nya meragukan.**
- **Saya akan menargetkan pasar di jabodetabek dan fokus ke kualitas produk**

---